# "Apport d'un modèle Bayésien lors de l'utilisation de réseaux de neuronnes pour la classification d'images"

Comparaison de 2 versions du modèle AlexNet pour la classification d'images, exemple sur un jeu d'images d'insectes.

## Démarche et difficultés

### Rappel de la démarche projet 

Notre projet porte sur des problématiques d'apprentissage profond avec des réseaux de neuronnes pour de la reconnaissance d'images. Il a pour but d'évaluer l'apport d'un modèle comportant des composantes bayésiennes par rapport à un modèle purement fréquentiste. En comparant les 2 modèles du point de vue de l'acuracy en fonction des ressources demandées (nombre d'image dans le jeu de donnée, temps d'entrainement du modèle, ressource GPU, ...) nous pourrons alors déterminer dans quels cas il est préférable d'utiliser l'une des 2 méthodes.

Concrêtement, il s'agira de prendre le modèle AlexNet déjà pré-entrainé (en conservant les poids optenu à la suite de son entrainement sur ImageNet) et de le fine-tuner pour nos données. Il en sera fait de même pour le modèle bayésien pour permettre une comparaison ayant le plus de sens possible.

### Problèmes encontrés

Pour ce projet, nous sommes passé par 3 étapes clés : 
- Mise en route : recherches documentaires/bibliographiques + établissement de la marche à suivre
- CNN : compréhension, mise en place et résultats
- BNN : compréhension, mise en place et résultats

Lors de nos travaux, nous nous somme heurté à 2 difficultés majeures :   
- Une difficulté de compréhension tout d'abord : Il nous fallait réellement et concrêtement comprendre les tenant et aboutissant d'un réseau de neurones pour pouvoir l'appliquer à notre cas.
- Une difficulté technique par la suite : Une fois le fonctionnement du modèle définit, il nous fallait le faire fonctionner. La compréhension des objets manipulés et des espaces de départs et d'arrvée nous ont tout particulièrement occupé.

Ces difficultés étaient attendu, et ont été dépassées pour l'étape CNN après un travail commun. Nous comptions ainsi faire de même pour la dernière étape : le BNN. Cependant, malgré les bases de la méthode solidement encrées et acquises via le travail sur le CNN, l'adaptation au cas du BNN ne s'est pas faites aussi facilement, ce qui était également attendu. Seulement pas à cette échelle. 

Opérationnelement, nous bloquons à obtenir un modèle avec une fonction de perte cohérente à un modèle entrainé et fine-tuné, et des performances dépassant le hazard. Cela traduit certainement un problèmes dans le calcul et la transmission des distributions de poids pour chaque couches.

### Objectifs de ce document 

Nos efforts ont certes drastiquement amélioré le modèle, mais ses performances restent insuffisants. Ainsi, les résultats restent très insatisfaisant et ne nous laissent pas l'opportunité de comparer pleinenemtn avec le modèle CNN malgré nos efforts. Nous avons donc pris la décision de compiler l'ensemble de notre travail à la fois de réflexion et de modification techniques pour tenter d'aboutir à un modèle utilisable, et ce pour mieux vous faire comprendre l'établissement de modèles d'apprentissages profonds en bayésiens, type BNN.

## Mise en oeuvre et changements

### Premier pas : package Pytorch bayesianCNN

### Modifications :

Modification 1 : 

La première difficulté que nous avons rencontré c'est qu'en bayésien les poids sont des distributions et non pas des nombres.
Donc les poids pré-entrainés de AlexNet sur ImageNet ne sont pas adapter.
Pour cela on a fait la fonction transfer_weights_to_bayesian.
Avec cette fonction, les poids sont choisis pour être des gaussienne centrées sur les poids d'AlexNet.

In [ ]:
# 3. Transférer les poids vers les moyennes (mu) des distributions bayésiennes
def transfer_weights_to_bayesian(bayesian_model, pretrained_model):
    """
    Transfère les poids d'un modèle classique vers les moyennes (mu) 
    des distributions bayésiennes
    """
    # Mapping des couches conv
    conv_mapping = {
        'conv1': 0,  # pretrained.features[0] -> bayesian.conv1
        'conv2': 3,  # pretrained.features[3] -> bayesian.conv2
        'conv3': 6,  # pretrained.features[6] -> bayesian.conv3
        'conv4': 8,  # pretrained.features[8] -> bayesian.conv4
        'conv5': 10, # pretrained.features[10] -> bayesian.conv5
    }
    
    # Transférer les couches de convolution
    for bay_name, pre_idx in conv_mapping.items():
        bay_layer = getattr(bayesian_model, bay_name)
        pre_layer = pretrained_model.features[pre_idx]
        
        # Copier weight vers weight_mu
        bay_layer.W_mu.data.copy_(pre_layer.weight.data)
        if pre_layer.bias is not None:
            bay_layer.bias_mu.data.copy_(pre_layer.bias.data)
        
        print(f"✅ Transféré {bay_name} depuis features[{pre_idx}]")
    
    print("✅ Transfert terminé ! Les moyennes bayésiennes sont initialisées avec AlexNet pré-entraîné")



Modification 2 :

Le problème avec cette solution, c'est qu'on avait une loss très importante par rapport à ce qu'on avait avec le CNN.
La loss était de l'ordre du milion et augmentait au cours des époches.
Pour corriger cela nous avons tester des priors différents. Nous avons retenu des petites valeurs de sigma.

In [ ]:
priors = {
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.01),  # plus petit écart pour mu
    'posterior_rho_initial': (-4, 0.01),  # plus petit écart pour rho → sigma initial plus faible
}


Modification 3 :

Nous avons mis un très petit learning rate car sinon la loss augmentait trop.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.0005)


Modification 4 : 

Nous avons fait un travail de normalisation en centrant, réduisant puis en appliquant une fonction arcsin à nos données pour réduire les valeurs extrêmes en ajoutant une borne à -3 et 3.
Sans ça, la loss était tellement grande qu'elle n'étant pas mesurable.

In [ ]:
# === Transformation commune ===
mean = [0.0, 0.0, 0.0]
std = [1.0, 1.0, 1.0]

class ArcSinhTransform:
    def __call__(self, x):
        return torch.asinh(x)

class ClipTransform:
    def __init__(self, min_val=-3.0, max_val=3.0):
        self.min_val = min_val
        self.max_val = max_val
    def __call__(self, x):
        return torch.clamp(x, self.min_val, self.max_val)

transform = transforms.Compose([
    transforms.Resize((189, 189)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    ArcSinhTransform(),
    ClipTransform(-3.0, 3.0)
])

Modification 5 :

On a beaucoup diminuer le poid du kl et on l'a rendu adaptatif. Si on ne faisait pas cela, la divergence de kullback-Liebler était trop importante et l'algorithme voulais juste minimiser ce critère.

In [ ]:
kl_weight = min(0.0005, (epoch + 1) * 0.0005 / 5)  # KL annealing
